In [1]:
''' IMPORTS '''

from pprint import pprint
import math
from datetime import datetime

import pandas as pd
import numpy as np

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_team_data, get_pbp_data


pio.templates['nfl_template'] = nfl_template

In [11]:
''' Constants / Parameters  '''

## Parameters
INPUT_YEARS = [i for i in range(2018, 2026)]

# NOTE - make sure not to include any un-completed matchups in model
SEASON = 2025
PREDICTION_WEEK = 9

FEATURE_TYPE = 'EPA / Play'

## Constants ## 

EPA_COLS = []
EPA_PLAY_COLS = []
for n in [4,8,12,16]:
    for unit in ['O', 'D', 'ST']:
        EPA_COLS.append(f'Last_{n}_EPA_{unit}')
        EPA_PLAY_COLS.append(f'Last_{n}_EPA_{unit}_Play')

FEATURE_COLS = EPA_PLAY_COLS if FEATURE_TYPE == 'EPA / Play' else EPA_COLS

# Process

In [ ]:
''' Import Data '''

# # Import
# print(f'Downloading pbp data from {YEARS[0]} to {YEARS[-1]}')
# pbp_data: pd.DataFrame = nfl.import_pbp_data(YEARS, downcast=True)

# # Some cleaning
# pbp_data = pbp_data.loc[pbp_data['season_type'] == 'REG', :]
# for col in ['home_team', 'away_team', 'posteam', 'defteam']:
#     pbp_data[col] = pbp_data[col].replace('OAK', 'LV')

# ## Success
# pbp_data['% ydstogo'] = pbp_data['yards_gained'] / pbp_data['ydstogo']

# pbp_data['Successful Play'] = (
#     ((pbp_data['down'] == 1) & (pbp_data['% ydstogo'] >= 0.4)) |
#     ((pbp_data['down'] == 2) & (pbp_data['% ydstogo'] >= 0.6)) |
#     (pbp_data['first_down'] == 1) |
#     (pbp_data['touchdown'] == 1)
# )

# ## Snaps
# pbp_data['Snap'] = ((pbp_data['pass'] == 1) | (pbp_data['rush']) == 1 | (pbp_data['special'] == 1))

# ## Yard Thresholds
# def distance_range(down, yds):
       
#     down_s = ''
#     match down:
#         case 1:
#             # down_s = '1st'
#             return '1st'
#         case 2:
#             down_s = '2nd'
#         case 3:
#             down_s = '3rd'
#         case 4:
#             down_s = '4th'
#         case default:
#             return ''
        
#     yds_range = ''
#     if yds <= 2:
#         yds_range = 'Short'
#     elif yds <= 6:
#         yds_range = 'Medium'
#     else:
#         yds_range = 'Long'

#     return f'{down_s} & {yds_range}'
    
# pbp_data['Down & Distance'] = pbp_data.apply(lambda x: distance_range(x['down'], x['ydstogo']), axis=1)

2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
2025 done.
Downcasting floats.


In [3]:
''' Import / Process Data '''

## Team info ##
team_data = get_team_data()

## PBP ##
pbp_data = get_pbp_data(years=INPUT_YEARS)

# Filter to current week
# pbp_filtered = pbp_data.loc[pbp_data['week'] <= PREDICTION_WEEK, :]

## Matchups ##
schedule_data = nfl.import_schedules(years=INPUT_YEARS)

# Some cleaning
schedule_data['winner'] = np.where(schedule_data['result'] > 0, 1, 0)
schedule_data = schedule_data.replace('OAK', 'LV')

# Get desired columns / weeks
COLS = ['game_id', 'season', 'week', 'home_team', 'away_team', 'result', 'winner']
FILTERS = (schedule_data['game_type'] == 'REG') & (schedule_data['result'] != 0)

master_matchups_df = schedule_data.loc[FILTERS, COLS].sort_values(by=['season', 'week']).reset_index(drop=True)

## Create week master ##
master_weeks = master_matchups_df[['season', 'week']].drop_duplicates().reset_index(drop=True)
master_weeks.index = master_weeks.index + 1
master_weeks = master_weeks.reset_index(names=['master_week'])

# Add week back to matchup
master_matchups_df = master_matchups_df.merge(master_weeks, left_on=['season', 'week'], right_on=['season', 'week'])

print(master_weeks.shape)
print(master_weeks.tail().to_string())
print(master_matchups_df.shape)
print(master_matchups_df.loc[master_matchups_df['master_week'] == master_matchups_df['master_week'].max(),:].to_string())

2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
2025 done.
Downcasting floats.


/Users/jmiller/Documents/Fun/nfl/notebooks/resources/get_nfl_data.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pbp_data['Non-Play Type'] = conditions
/Users/jmiller/Documents/Fun/nfl/notebooks/resources/get_nfl_data.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pbp_data['Offensive Snap'] = (((pbp_data['pass'] == 1) | (pbp_data['rush'] == 1)) & (~pbp_data['epa'].isna()))
/Users/jmiller/Documents/Fun/nfl/notebooks/resources/get_nfl_data.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

(141, 3)
     master_week  season  week
136          137    2025    14
137          138    2025    15
138          139    2025    16
139          140    2025    17
140          141    2025    18
(2119, 8)
              game_id  season  week home_team away_team  result  winner  master_week
2103   2025_18_NO_ATL    2025    18       ATL        NO     NaN       0          141
2104  2025_18_NYJ_BUF    2025    18       BUF       NYJ     NaN       0          141
2105  2025_18_DET_CHI    2025    18       CHI       DET     NaN       0          141
2106  2025_18_CLE_CIN    2025    18       CIN       CLE     NaN       0          141
2107  2025_18_LAC_DEN    2025    18       DEN       LAC     NaN       0          141
2108  2025_18_IND_HOU    2025    18       HOU       IND     NaN       0          141
2109  2025_18_TEN_JAX    2025    18       JAX       TEN     NaN       0          141
2110   2025_18_ARI_LA    2025    18        LA       ARI     NaN       0          141
2111    2025_18_KC_LV    2025 

In [4]:
''' Calculate Weekly EPA '''

# PLAY_TYPES = ['GAME_START', 'KICK_OFF', 'PENALTY', 'PASS', 'RUSH', 'PUNT', 'FIELD_GOAL', 'SACK',\
#             'END_QUARTER', 'TIMEOUT', 'UNSPECIFIED', 'XP_KICK', 'INTERCEPTION', 'PAT2', 'END_GAME', \
#             'COMMENT', 'FUMBLE_RECOVERED_BY_OPPONENT', 'FREE_KICK']
# PLAY_TYPES_SPECIAL = ['KICK_OFF', 'PUNT', 'FIELD_GOAL', 'XP_KICK']
# NON_PLAY_TYPES = ['GAME_START','END_QUARTER', 'TIMEOUT', 'END_GAME', 'UNSPECIFIED', 'COMMENT', 'FREE_KICK']

# # Do global filtering
# conditions = (~pbp_data['play_type'].isna()) &\
#             (~pbp_data['play_type'].isin(['qb_kneel', 'qb_spike'])) &\
#             (pbp_data['timeout'] == 0) &\
#             (~pbp_data['play_type_nfl'].isin(NON_PLAY_TYPES))
# pbp_filtered = pbp_data.loc[conditions, :].copy()


# special_conditions = (pbp_filtered['play_type_nfl'].isin(PLAY_TYPES_SPECIAL)) |\
#                       (pbp_filtered['special_teams_play'] == 1)                     #(pbp_data['special_teams_play'] == 0) &\

offense_epa = pbp_data.loc[pbp_data['Is Special Teams Play'] == False, :].groupby(['season', 'week', 'posteam']).aggregate(
    Plays_O=('posteam', 'size'),
    EPA_O=('epa', 'sum')
)#.reset_index().rename(columns={'posteam': 'team'})
offense_epa['EPA_O_Play'] = offense_epa['EPA_O'] / offense_epa['Plays_O']
offense_epa.index = offense_epa.index.set_names('team', level=2)

## TODO - rename defteam to team or posteam
defense_epa = pbp_data.loc[pbp_data['Is Special Teams Play'] == False, :].groupby(['season', 'week', 'defteam']).aggregate(
    Plays_D=('posteam', 'size'),
    EPA_D=('epa', 'sum')
)#.reset_index().rename(columns={'defteam': 'team'})
defense_epa['EPA_D'] = -1 * defense_epa['EPA_D']
defense_epa['EPA_D_Play'] = defense_epa['EPA_D'] / defense_epa['Plays_D']
defense_epa.index = defense_epa.index.set_names('team', level=2)

## TODO - total up offensive and defensive epa per team
special_teams_epa = pbp_data.loc[pbp_data['Is Special Teams Play'] == True, :].groupby(['season', 'week', 'posteam']).aggregate(
    Opp=('defteam', 'first'),
    POS_Plays_ST=('posteam', 'size'),
    POS_EPA_ST=('epa', 'sum')
)

def get_def_plays(row):
    seas = row.name[0]
    w = row.name[1]
    opp = row['Opp']
    return special_teams_epa.loc[(seas, w, opp), 'POS_Plays_ST']

def get_def_epa(row):
    seas = row.name[0]
    w = row.name[1]
    opp = row['Opp']
    return -1*special_teams_epa.loc[(seas, w, opp), 'POS_EPA_ST']

special_teams_epa['DEF_Plays_ST'] = special_teams_epa.apply(lambda x: get_def_plays(x), axis=1)
special_teams_epa['DEF_EPA_ST'] = special_teams_epa.apply(lambda x: get_def_epa(x), axis=1)

special_teams_epa['Plays_ST'] = special_teams_epa['POS_Plays_ST'] + special_teams_epa['DEF_Plays_ST']
special_teams_epa['EPA_ST'] = special_teams_epa['POS_EPA_ST'] + special_teams_epa['DEF_EPA_ST']
special_teams_epa['EPA_ST_Play'] = special_teams_epa['EPA_ST'] / special_teams_epa['Plays_ST']

special_teams_epa.index = special_teams_epa.index.set_names('team', level=2)
# special_teams_epa = special_teams_epa.reset_index().rename(columns={'posteam': 'team'})

# print(offense_epa.head().to_string())
# print(defense_epa.head().to_string())
# print(special_teams_epa.head().to_string())

master_epa_df = offense_epa.merge(defense_epa, left_index=True, right_index=True)
master_epa_df = master_epa_df.merge(special_teams_epa, left_index=True, right_index=True).reset_index()

master_epa_df = master_epa_df.merge(master_weeks, left_on=['season', 'week'], right_on=['season', 'week'], how='left')
print(master_epa_df.tail().to_string())


      season  week team  Plays_O      EPA_O  EPA_O_Play  Plays_D      EPA_D  EPA_D_Play  Opp  POS_Plays_ST  POS_EPA_ST  DEF_Plays_ST  DEF_EPA_ST  Plays_ST     EPA_ST  EPA_ST_Play  master_week
3947    2025     8  PIT       63  -7.043552   -0.111802       66 -17.920689   -0.271526   GB            17    8.390466            18    3.705885        35  12.096351     0.345610          131
3948    2025     8   SF       48  -2.852764   -0.059433       83 -16.417570   -0.197802  HOU            14    5.109242            11   -1.205309        25   3.903934     0.156157          131
3949    2025     8   TB       63 -10.183881   -0.161649       76  34.452625    0.453324   NO            10    4.498999            10   -0.611090        20   3.887909     0.194395          131
3950    2025     8  TEN       73  -2.045314   -0.028018       54 -18.318108   -0.339224  IND            15   -5.787210            13   -1.579212        28  -7.366421    -0.263086          131
3951    2025     8  WAS       55  -8.933

In [5]:
''' Reshape dfs '''

master_weeks = master_weeks.set_index(['season', 'week'])
master_epa_df = master_epa_df.set_index(['master_week', 'team'])

print(master_weeks.tail().to_string())
print(master_matchups_df.loc[master_matchups_df['master_week'] == 129,:].to_string())
print(master_epa_df.loc[master_epa_df.index.get_level_values(0) == 129,:].to_string())

             master_week
season week             
2025   14            137
       15            138
       16            139
       17            140
       18            141
              game_id  season  week home_team away_team  result  winner  master_week
1925  2025_06_PHI_NYG    2025     6       NYG       PHI    17.0       1          129
1926  2025_06_DEN_NYJ    2025     6       NYJ       DEN    -2.0       0          129
1927   2025_06_LA_BAL    2025     6       BAL        LA   -14.0       0          129
1928  2025_06_DAL_CAR    2025     6       CAR       DAL     3.0       1          129
1929  2025_06_ARI_IND    2025     6       IND       ARI     4.0       1          129
1930  2025_06_SEA_JAX    2025     6       JAX       SEA    -8.0       0          129
1931  2025_06_LAC_MIA    2025     6       MIA       LAC    -2.0       0          129
1932    2025_06_NE_NO    2025     6        NO        NE    -6.0       0          129
1933  2025_06_CLE_PIT    2025     6       PIT       CLE    1

In [8]:

def get_epa_inputs(teams: list, master_week: int):
    
    # Start return df
    teams_df = pd.DataFrame(data={'team': teams}).set_index('team')

    # Sum up EPA and Plays for each team and last n games
    for team in teams:
        team_sl = master_epa_df.loc[master_epa_df.index.get_level_values(1) == team, :]

        for n in [4,8,12,16]:
            sl = team_sl.loc[(team_sl.index.get_level_values(0) < master_week),:].tail(n)
            if team == 'IND' and n == 4 and master_week == 132:
                print(sl.head().to_string())
                
            for unit in ['O', 'D', 'ST']:
                epa = sl[f'EPA_{unit}'].sum()
                plays = sl[f'Plays_{unit}'].sum()

                teams_df.loc[team, f'Last_{n}_EPA_{unit}'] = epa
                teams_df.loc[team, f'Last_{n}_EPA_{unit}_Play'] = epa / plays

    teams_df = teams_df.reset_index()

    return teams_df

# # Params
# season = 2025
# week = 9
# team = 'IND'
# n = 4

# # Go
# c_master_week = master_weeks.loc[(season, week), 'master_week']
# print(c_master_week)

# results = get_epa_inputs(['IND', 'MIA'], c_master_week)
# print(results.to_string())

In [9]:
''' Forge Historical Weekly EPA Inputs for Historical Matchups '''


c_master_week = master_weeks.loc[(SEASON, PREDICTION_WEEK), 'master_week']
print(c_master_week)

## Matchups
input_weeks = master_weeks.loc[(master_weeks.index.get_level_values(0) >= 2019) & (master_weeks['master_week'] < c_master_week), 'master_week'].unique().tolist()
input_weeks: list[int] = master_weeks.loc[master_weeks.index.get_level_values(0) >= 2019, 'master_week'].unique().tolist()
print(input_weeks)

input_matchups = master_matchups_df.loc[master_matchups_df['master_week'].isin(input_weeks),:]
# input_matchups.loc[input_matchups['game_id'] == '2025_06_ARI_IND', ['home_team', 'away_team']] = ['ARI', 'IND']

## EPA Inputs

epa_inputs_df = pd.DataFrame(columns=['master_week', 'team'] + EPA_COLS + EPA_PLAY_COLS)

for week in input_weeks:

    home_teams = input_matchups.loc[input_matchups['master_week'] == week, 'home_team'].unique().tolist()
    away_teams = input_matchups.loc[input_matchups['master_week'] == week, 'away_team'].unique().tolist()
    
    df = get_epa_inputs(home_teams+away_teams, master_week=week)
    df['master_week'] = week

    epa_inputs_df = pd.concat([epa_inputs_df, df])

epa_inputs_df = epa_inputs_df.reset_index(drop=True)

## Home team EPA
rename_dict = {col: f'Home_Team_{col}' for col in EPA_COLS + EPA_PLAY_COLS}
input_matchups = input_matchups.merge(epa_inputs_df, left_on=['master_week', 'home_team'], right_on=['master_week', 'team'], how='left').rename(columns=rename_dict).drop(columns='team')

## Away team EPA
rename_dict = {col: f'Away_Team_{col}' for col in EPA_COLS + EPA_PLAY_COLS}
input_matchups = input_matchups.merge(epa_inputs_df, left_on=['master_week', 'away_team'], right_on=['master_week', 'team'], how='left').rename(columns=rename_dict).drop(columns='team')


print(input_matchups.loc[input_matchups['master_week'] == c_master_week, :].to_string())

131
[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141]
                  season  week  Plays_O      EPA_O  EPA_O_Play  Plays_D      EPA_D  EPA_D_Play  Opp  POS_Plays_ST  POS_EPA_ST  DEF_Plays_ST  DEF_EPA_ST  Plays_ST    EPA_ST  EPA_ST_Play
master_week team                                                                                                                                                                        
128         IND     2025     5       64  20.770067    0.324532       70  13.344496    0

# Modeling

In [12]:
''' Model Parameters and Final Prep '''

# Last Completed Week
last_completed_week_master = master_weeks.loc[(SEASON, PREDICTION_WEEK - 1), 'master_week']
print(f'Last completed weeK:', last_completed_week_master)

# Features
FEATURES = [f'Home_Team_{col}' for col in FEATURE_COLS] + [f'Away_Team_{col}' for col in FEATURE_COLS]
print(FEATURES)

# Matchups w EPA
input_matchups_sl = input_matchups.loc[input_matchups['master_week'] <= last_completed_week_master, :]

sl = input_matchups_sl.loc[input_matchups_sl['master_week'] == input_matchups_sl['master_week'].min(), ['season', 'week', 'master_week']].drop_duplicates()
print(f'First input week: {sl["season"].values[0]}, Week {sl["week"].values[0]}')
sl = input_matchups_sl.loc[input_matchups_sl['master_week'] == input_matchups_sl['master_week'].max(), ['season', 'week', 'master_week']].drop_duplicates()
print(f'Last input week: {sl["season"].values[0]}, Week {sl["week"].values[0]}')

print(input_matchups_sl.loc[input_matchups_sl['master_week'] == sl['master_week'].values[0], :].to_string())

Last completed weeK: 131
['Home_Team_Last_4_EPA_O_Play', 'Home_Team_Last_4_EPA_D_Play', 'Home_Team_Last_4_EPA_ST_Play', 'Home_Team_Last_8_EPA_O_Play', 'Home_Team_Last_8_EPA_D_Play', 'Home_Team_Last_8_EPA_ST_Play', 'Home_Team_Last_12_EPA_O_Play', 'Home_Team_Last_12_EPA_D_Play', 'Home_Team_Last_12_EPA_ST_Play', 'Home_Team_Last_16_EPA_O_Play', 'Home_Team_Last_16_EPA_D_Play', 'Home_Team_Last_16_EPA_ST_Play', 'Away_Team_Last_4_EPA_O_Play', 'Away_Team_Last_4_EPA_D_Play', 'Away_Team_Last_4_EPA_ST_Play', 'Away_Team_Last_8_EPA_O_Play', 'Away_Team_Last_8_EPA_D_Play', 'Away_Team_Last_8_EPA_ST_Play', 'Away_Team_Last_12_EPA_O_Play', 'Away_Team_Last_12_EPA_D_Play', 'Away_Team_Last_12_EPA_ST_Play', 'Away_Team_Last_16_EPA_O_Play', 'Away_Team_Last_16_EPA_D_Play', 'Away_Team_Last_16_EPA_ST_Play']
First input week: 2019, Week 1
Last input week: 2025, Week 8
              game_id  season  week home_team away_team  result  winner master_week  Home_Team_Last_4_EPA_O  Home_Team_Last_4_EPA_D  Home_Team_Last_4

In [13]:
''' Vizualize Inputs '''

## Feature Correlation Matrix
corr = input_matchups[['result'] + FEATURES].corr()

fig = px.imshow(
    corr,
    template='nfl_template',
    title='Features Correlation Matrix',
    zmin=-1,
    zmax=1,
    text_auto=True,
    # aspect='auto'
)
fig.show()

## Feature Distributions
for unit in ['O', 'D', 'ST']:
    col = f'Home_Team_Last_16_EPA_{unit}'
    fig = px.histogram(
        template='nfl_template',
        x=input_matchups[col],
        title=f'{col}<br><sup>Distribution</sup>'
    )
    fig.show()

## Win Probability - Logistic Regression

In [14]:
''' Logistic regression - Win Probability '''


# Get X and y
X = input_matchups_sl[FEATURES].to_numpy()
y = input_matchups_sl['winner'].to_numpy()
print(X.shape)
print(y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the data (Standardization).
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Create a Logistic Regression model
log_reg_model = LogisticRegression(max_iter=100, solver='liblinear') # Increased max_iter for convergence

# Train the model
log_reg_model.fit(X_train, y_train)

# Make predictions
y_pred = log_reg_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:,.2f}%")

# coefs_dict = {FEATURES[i]: log_reg_model.coef_[0][i] for i in range(len(FEATURES))}
# pprint(coefs_dict, sort_dicts=False)

(1714, 24)
(1714,)
Accuracy: 65.24%


In [53]:
for c in range(3):
    fig = px.histogram(
        x=[X[i][c] for i in range(len(X))],
    )
    fig.show()

In [ ]:
''' Graph for Overfitting / Underfitting '''

## Spreads - Linear Regression

In [15]:
''' Linear Regression - Spread '''

# Get X and y
X = input_matchups_sl[FEATURES].to_numpy()
y = input_matchups_sl['result'].to_numpy()
print(X.shape)
print(y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the data (Standardization).
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Create a Logistic Regression model
lin_reg_model = LinearRegression() # Increased max_iter for convergence

# Train the model
lin_reg_model.fit(X_train, y_train)

# Make predictions
y_pred = lin_reg_model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
# Print evaluation metrics
print(f"R-squared: {r2:.4f}")
print(f"Mean squared error: {mse:.4f}")
print(f"Root mean squared error: {rmse:.4f}")

(1714, 24)
(1714,)
R-squared: 0.1724
Mean squared error: 183.0816
Root mean squared error: 13.5308


## Predictions

In [16]:
''' Predict a week '''

# Find matchups
c_master_week = master_weeks.loc[(SEASON, PREDICTION_WEEK), 'master_week']
matchups = input_matchups.loc[input_matchups['master_week'] == c_master_week, :]

print(c_master_week)
print(matchups.to_string())

# Get inputs
X = matchups[FEATURES].to_numpy()

## WIN PROBABILITY
y_pred = log_reg_model.predict(X)
probs = log_reg_model.predict_proba(X)

## SPREAD
results = lin_reg_model.predict(X)

## Collect results
predictions_df = matchups[['game_id', 'season', 'week', 'home_team', 'away_team']].copy().reset_index(drop=True)

# Win Probs
predictions_df['prob_home'] = [probs[i][1] for i in range(len(probs))]
predictions_df['prob_away'] = [probs[i][0] for i in range(len(probs))]
predictions_df['pred'] = np.where(y_pred == 1, predictions_df['home_team'], predictions_df['away_team'])
predictions_df['pred_prob'] = predictions_df[['prob_home','prob_away']].max(axis=1)

# Spread
predictions_df['spread'] = results

print(predictions_df.to_string())


132
              game_id  season  week home_team away_team  result  winner master_week  Home_Team_Last_4_EPA_O  Home_Team_Last_4_EPA_D  Home_Team_Last_4_EPA_ST  Home_Team_Last_8_EPA_O  Home_Team_Last_8_EPA_D  Home_Team_Last_8_EPA_ST  Home_Team_Last_12_EPA_O  Home_Team_Last_12_EPA_D  Home_Team_Last_12_EPA_ST  Home_Team_Last_16_EPA_O  Home_Team_Last_16_EPA_D  Home_Team_Last_16_EPA_ST  Home_Team_Last_4_EPA_O_Play  Home_Team_Last_4_EPA_D_Play  Home_Team_Last_4_EPA_ST_Play  Home_Team_Last_8_EPA_O_Play  Home_Team_Last_8_EPA_D_Play  Home_Team_Last_8_EPA_ST_Play  Home_Team_Last_12_EPA_O_Play  Home_Team_Last_12_EPA_D_Play  Home_Team_Last_12_EPA_ST_Play  Home_Team_Last_16_EPA_O_Play  Home_Team_Last_16_EPA_D_Play  Home_Team_Last_16_EPA_ST_Play  Away_Team_Last_4_EPA_O  Away_Team_Last_4_EPA_D  Away_Team_Last_4_EPA_ST  Away_Team_Last_8_EPA_O  Away_Team_Last_8_EPA_D  Away_Team_Last_8_EPA_ST  Away_Team_Last_12_EPA_O  Away_Team_Last_12_EPA_D  Away_Team_Last_12_EPA_ST  Away_Team_Last_16_EPA_O  Away_Tea

# Visualize Predictions

In [17]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')  # Remove '#' if present
    
    # Extract the red, green, and blue components
    r_hex = hex_color[0:2]
    g_hex = hex_color[2:4]
    b_hex = hex_color[4:6]
    
    # Convert hexadecimal to decimal integers
    r = int(r_hex, 16)
    g = int(g_hex, 16)
    b = int(b_hex, 16)
    
    return (r, g, b)


print(hex_to_rgb('#003b75'))

(0, 59, 117)


In [18]:
''' Format Results for Visualization '''

viz_df = predictions_df.copy()

# Add betting odds
viz_df['home_team_ml'] = np.where(viz_df['prob_home'] > viz_df['prob_away'],
                                  (-100*viz_df['prob_home'])/(1 - viz_df['prob_home']),
                                  ((1 - viz_df['prob_home'])/viz_df['prob_home'])*100).astype(int)
viz_df.loc[viz_df['home_team_ml'] > 0, 'home_team_ml'] = '+' + viz_df['home_team_ml'].astype(str)

viz_df['away_team_ml'] = np.where(viz_df['prob_away'] > viz_df['prob_home'],
                                  (-100*viz_df['prob_away'])/(1 - viz_df['prob_away']),
                                  ((1 - viz_df['prob_away'])/viz_df['prob_away'])*100).astype(int)
viz_df.loc[viz_df['away_team_ml'] > 0, 'away_team_ml'] = '+' + viz_df['away_team_ml'].astype(str)


# Get logos and colors
logos_df = pd.read_csv('../data/NFL_teams_info.csv', usecols=['abbreviation', 'color', 'logoURL'])
logos_df.loc[logos_df['abbreviation'] == 'LAR', 'abbreviation'] = 'LA'
logos_df.loc[logos_df['abbreviation'] == 'WSH', 'abbreviation'] = 'WAS'
logos_df['color'] = '#' + logos_df['color']

viz_df = viz_df.merge(logos_df, left_on='home_team', right_on='abbreviation', how='left').drop(columns=['abbreviation']).rename(columns={'color': 'home_color', 'logoURL': 'home_logo'})
viz_df = viz_df.merge(logos_df, left_on='away_team', right_on='abbreviation', how='left').drop(columns=['abbreviation']).rename(columns={'color': 'away_color', 'logoURL': 'away_logo'})
print(viz_df.to_string())


            game_id  season  week home_team away_team  prob_home  prob_away pred  pred_prob     spread home_team_ml away_team_ml home_color                                          home_logo away_color                                          away_logo
0   2025_09_BAL_MIA    2025     9       MIA       BAL   0.446689   0.553311  BAL   0.553311  -1.815155         +123         -123    #008e97  https://a.espncdn.com/i/teamlogos/nfl/500/mia.png    #29126f  https://a.espncdn.com/i/teamlogos/nfl/500/bal.png
1   2025_09_CHI_CIN    2025     9       CIN       CHI   0.600032   0.399968  CIN   0.600032   5.213673         -150         +150    #fb4f14  https://a.espncdn.com/i/teamlogos/nfl/500/cin.png    #0b1c3a  https://a.espncdn.com/i/teamlogos/nfl/500/chi.png
2   2025_09_MIN_DET    2025     9       DET       MIN   0.697732   0.302268  DET   0.697732   5.111326         -230         +230    #0076b6  https://a.espncdn.com/i/teamlogos/nfl/500/det.png    #4f2683  https://a.espncdn.com/i/teamlogos/nfl/

In [22]:
''' Win Probability - Pie Charts '''

# Create pie chart for each game
titles = []
pie_charts = []
for i in viz_df.index:

    away_prob, home_prob = viz_df.loc[i, ['prob_away', 'prob_home']]
    away_team, home_team = viz_df.loc[i, ['away_team', 'home_team']]
    
    away_color, home_color = viz_df.loc[i, ['away_color', 'home_color']]
    away_logo, home_logo = viz_df.loc[i, ['away_logo', 'home_logo']]

    # away_rbg = hex_to_rgb(away_color)
    # away_rbg = f'rgba({away_rbg[0]},{away_rbg[1]},{away_rbg[2]},{1 if away_prob > home_prob else 0.5})'

    # home_rbg = hex_to_rgb(home_color)
    # home_rbg = f'rgba({home_rbg[0]},{home_rbg[1]},{home_rbg[2]},{1 if home_prob > away_prob else 0.5})'
    # # away_rbg = (*away_rbg, 1)
    # print(away_team, away_rbg)

    off_black = 'rgba(0,0,0,0.7)'
    pie_chart = go.Pie(
        values=[home_prob, away_prob],
        labels=[home_team, away_team],
        marker=dict(
            colors=[home_color if home_prob > away_prob else off_black, away_color if away_prob > home_prob else off_black], 
            pattern=dict(
                shape=["" if home_prob > away_prob else "x", "" if away_prob > home_prob else "x"],
                size=4,
            ),
            line=dict(color='#f3f3f3', width=2),
        ),
        textposition='outside',
        textinfo='percent+label',
        hole=0.4,
        textfont=dict(weight='bold'),
        sort=False
    )

    pie_charts.append(pie_chart)
    titles.append(f'{away_team} vs. {home_team}')

# Create Figure
N_COLS = 4
N_ROWS = math.ceil(len(viz_df) / N_COLS)
domain = [[{"type": "domain"} for i in range(N_COLS)] for i in range(N_ROWS)]

fig = make_subplots(rows=N_ROWS, cols=N_COLS, specs=domain, subplot_titles=titles, horizontal_spacing=0.05)

# Add charts to plot
i = 0
for r in range(1, N_ROWS+1):
    for c in range(1, N_COLS+1):
        fig.add_trace(
            pie_charts[i],
            row=r,
            col=c
        )

        i += 1
        if i > len(viz_df.index) - 1:
            break

fig.for_each_annotation(lambda a: a.update(font=dict(size=14, weight='bold')))

# fig.add_annotation(
#     # source='https://a.espncdn.com/i/teamlogos/nfl/500/jax.png',  # The loaded image
#     text='Heyyy',
#     xref="x2 domain", yref="y2 domain",
#     x=0.5, y=0.5, # Center of the plot area
#     # sizex=0.2, sizey=0.2, # Adjust size as needed
#     xanchor="center", yanchor="middle",
#     # layer="above", # Place image above other plot elements
#     # opacity=0.85,
#     row=2, col=2
# )


fig.update_layout(
    template='nfl_template',
    title=dict(
        text=f'NFL Week {PREDICTION_WEEK} <span style="color: #D5A15D">Win Probability</span><br><sup>EPA / Play Model</sup>',
    ),
    margin=dict(t=100, b=50, l=50, r=50),
    showlegend=False,
    height=900,
    width=900,
)

# Credits
fig.add_annotation(
    text=f'EPA / Play from teams\' last 4, 8, 12, and 16 games, in all 3 phases<br>Figure & Model: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    # font=dict(weight='bold'),
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.05, 
    x=1,
    align='right'
)
fig.show()


# Export
pio.write_image(fig, f'Week {PREDICTION_WEEK} Win Probability.png', scale=6, width=900, height=900)


In [27]:
''' Predictions '''

from PIL import Image
import requests
from io import BytesIO


BORDER_COLOR = '#989898'

tables = []
winner_logos = []
for i in viz_df.index:
    
    away_team, home_team = viz_df.loc[i, ['away_team', 'home_team']]
    away_prob, home_prob = viz_df.loc[i, ['prob_away', 'prob_home']]
    away_team_ml, home_team_ml = viz_df.loc[i, ['away_team_ml', 'home_team_ml']]
    
    spread = viz_df.loc[i, 'spread']
    spread = round(spread, 1)
    away_spread = f'+{abs(spread)}' if spread > 0 else f'-{abs(spread)}'
    home_spread = f'-{abs(spread)}' if spread > 0 else f'+{abs(spread)}'

    away_color, home_color = viz_df.loc[i, ['away_color', 'home_color']]
    away_logo, home_logo = viz_df.loc[i, ['away_logo', 'home_logo']]

    # TODO eventualy, Appr. Line
    table = go.Table(
        # columnwidth=[2,3,3],
        header=dict(
            values=['', 'Spread', 'Moneyline'],
            line_color=['rgba(0,0,0,0)']+[BORDER_COLOR]*3,
            fill_color=['rgba(0,0,0,0)']+['white']*3,
            align=['center', 'center'],
            font=dict(size=10)
        ),
        cells=dict(
            values=[[away_team, home_team], 
                    # [f'{away_prob*100:,.1f}%', f'{home_prob*100:,.1f}%'], 
                    [away_spread, home_spread], 
                    [away_team_ml, home_team_ml]],
            line_color=[BORDER_COLOR]*4,
            line_width=1,
            fill_color=[[away_color, home_color]]+['white']*3,
            font=dict(
                color=['white', 'black', 'black', 'black']
            )
        )
    )
    tables.append(table)
    winner_logos.append(away_logo if away_prob > home_prob else home_logo)


## Make Figure
N_ROWS = math.ceil(len(tables) / 2)
N_COLS = 2
fig = make_subplots(rows=N_ROWS, cols=N_COLS*2, 
                    specs=[[{"type": "table"}, {"type": "xy"}]*N_COLS for i in range(N_ROWS)],
                    subplot_titles=['', 'Projected<br>Winner']*len(tables),
                    horizontal_spacing=0.075,
                    column_widths=[5,1,5,1])

# Add tables / winner logos
i = 0
for r in range(1,N_ROWS+1):
    for c in range(1,N_COLS+1):

        ## Logo
        response = requests.get(winner_logos[i])
        img = Image.open(BytesIO(response.content))
        # img = img.resize((img.width*2, img.height*2), Image.LANCZOS)

        logo_trace = px.imshow(img=img)
        fig.add_trace(logo_trace.data[0], row=r, col=c*2)

        ## Table
        fig.add_trace(
            tables[i],
            row=r,col=(c*2)-1
        )

        i += 1
        if i > len(tables) - 1:
            break

    
## Formatting
fig.update_annotations(font=dict(size=10, weight='bold'))

fig.update_layout(
    template='nfl_template',
    title=dict(
        text=f'NFL Week {PREDICTION_WEEK} <span style="color: #D5A15D">Odds</span><br><sup>EPA / Play Model</sup>',
        y=0.965
    ),
    margin=dict(b=50, l=50, r=50),
    height=1000,
    width=700
)
fig.update_xaxes(
    visible=False,
)
fig.update_yaxes(
    visible=False,
)

# Credits
fig.add_annotation(
    text=f'EPA / Play from teams\' last 4, 8, 12, and 16 games, in all 3 phases<br>Figure & Model: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    font=dict(size=8),
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.05, 
    x=1,
    align='right'
)

fig.show()

# Export
pio.write_image(fig, f'Week {PREDICTION_WEEK} Predictions.png', scale=6, width=700, height=1000)

## (After Week) Results

In [62]:
print(master_matchups_df.head().to_string())

           game_id  season  week home_team away_team  result  winner  master_week
0  2018_01_ATL_PHI    2018     1       PHI       ATL     6.0       1            1
1  2018_01_BUF_BAL    2018     1       BAL       BUF    44.0       1            1
2  2018_01_CIN_IND    2018     1       IND       CIN   -11.0       0            1
3  2018_01_TEN_MIA    2018     1       MIA       TEN     7.0       1            1
4   2018_01_SF_MIN    2018     1       MIN        SF     8.0       1            1


In [63]:
## Init 

week_results_df = viz_df.merge(master_matchups_df[['game_id', 'winner']], on='game_id', how='left')
week_results_df['winner'] = np.where(week_results_df['winner'] == 0, week_results_df['away_team'], week_results_df['home_team'])

## Add Actual EPA
cols = ['EPA_O_Play', 'EPA_D_Play', 'EPA_ST_Play']

week_epa = master_epa_df.loc[master_epa_df.index.get_level_values(0) == c_master_week, :].reset_index()
print(week_epa.head().to_string())

# Home Team EPA
week_results_df = week_results_df.merge(week_epa[['team'] + cols], left_on='home_team', right_on='team', how='inner').rename(columns={
    col: f'home_team_{col}' for col in cols
}).drop(columns=['team'])

# Away Team EPA
week_results_df = week_results_df.merge(week_epa[['team'] + cols], left_on='away_team', right_on='team', how='inner').rename(columns={
    col: f'away_team_{col}' for col in cols
}).drop(columns=['team'])

week_results_df = week_results_df.drop(columns=['pred_prob', 'home_color', 'home_logo', 'away_color', 'away_logo'])
print(week_results_df.head().to_string())
week_results_df.to_excel('Week Results.xlsx', index=False, sheet_name='Results')

   master_week team  season  week  Plays_O      EPA_O  EPA_O_Play  Plays_D      EPA_D  EPA_D_Play  Opp  POS_Plays_ST  POS_EPA_ST  DEF_Plays_ST  DEF_EPA_ST  Plays_ST    EPA_ST  EPA_ST_Play
0          131  ATL    2025     8       51 -15.345836   -0.300899       70  -9.341197   -0.133446  MIA            13    0.131351            12   -3.786344        25 -3.654993    -0.146200
1          131  BAL    2025     8       65  13.515374    0.207929       65  -1.110908   -0.017091  CHI            13    4.606741            14   -2.332844        27  2.273897     0.084218
2          131  BUF    2025     8       65  20.989214    0.322911       61  19.096712    0.313061  CAR            13    1.330026            15   -2.685569        28 -1.355543    -0.048412
3          131  CAR    2025     8       61 -19.096712   -0.313061       65 -20.989214   -0.322911  BUF            15    2.685569            13   -1.330026        28  1.355543     0.048412
4          131  CHI    2025     8       65   1.110908    0.0